In [1]:
import numpy as np
import sympy as sp
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

In [2]:
# Definir variables de entrada
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6')
x = sp.Matrix([x1, x2, x3, x4, x5, x6])

# Definir pesos y sesgos como variables simbólicas
W1 = sp.MatrixSymbol('W1', 6, 872)
b1 = sp.MatrixSymbol('b1', 872, 1)
W2 = sp.MatrixSymbol('W2', 872, 864)
b2 = sp.MatrixSymbol('b2', 864, 1)
W3 = sp.MatrixSymbol('W3', 864, 1)
b3 = sp.MatrixSymbol('b3', 1, 1)

In [3]:
# Definir funciones de activación simbólicas
def relu(x):
    return sp.Piecewise((0, x <= 0), (x, x > 0))

def sigmoid(x):
    return 1 / (1 + sp.exp(-x))

def linear(x):
    return x

In [4]:
# Propagación hacia adelante simbólica
z1 = W1.T * x + b1
a1 = z1.applyfunc(relu)

z2 = W2.T * a1 + b2
a2 = z2.applyfunc(sigmoid)

z3 = W3.T * a2 + b3
output = linear(z3)

In [5]:
# Mostrar la función de salida simbólica
output

b3 + W3.T*Lambda(_d, 1/(1 + exp(-_d))).(b2 + W2.T*Lambda(_d, Piecewise((0, _d <= 0), (_d, True))).(b1 + W1.T*Matrix([
[x1],
[x2],
[x3],
[x4],
[x5],
[x6]])))

# Prediciendo salidas

In [6]:
# Cargar el modelo y definir la función neural_network_function como antes
model = load_model('../best_model_2.4367305027108534e-05.h5')
# Cargar los escaladores
scaler_X = joblib.load('../minmax_scaler_X.pkl')
scaler_y = joblib.load('../minmax_scaler_y.pkl')

W1, b1 = model.layers[0].get_weights()
W2, b2 = model.layers[1].get_weights()
W3, b3 = model.layers[2].get_weights()

c:\Users\PcR\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def neural_network_function(input_data):
    z1 = np.dot(input_data, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    z3 = np.dot(a2, W3) + b3
    return z3[0]

In [8]:
# Cargar el Excel con las predicciones
df = pd.read_excel('../matriz_datos_con_predicciones.xlsx')
X_normalized = scaler_X.transform(X)

# Extraer las columnas de entrada y la columna de predicciones
input_columns = ['y1', 'G_', 'wt_mea', 'z', 'T_gas', 'L_']
X = df[input_columns].values
y_pred_original = df['Kg RED'].values

NameError: name 'X' is not defined

In [ ]:
# Normalizar los datos de entrada
X_normalized = scaler_X.transform(X)

# Aplicar la función neural_network_function a los datos normalizados
y_pred_new_normalized = np.array([neural_network_function(x.reshape(1, -1)) for x in X_normalized])

c:\Users\PcR\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Comparación:
En el código, estamos comparando tres conjuntos de predicciones:

Las predicciones originales que ya estaban en tu archivo Excel.
Las nuevas predicciones hechas con nuestra función neural_network_function.
Las predicciones hechas directamente con el modelo cargado de Keras/TensorFlow.

In [ ]:
# Desnormalizar las predicciones
y_pred_new = scaler_y.inverse_transform(y_pred_new_normalized.reshape(-1, 1)).flatten()

# Calcular el error cuadrático medio
mse = np.mean((y_pred_original - y_pred_new)**2)
print(f"Error cuadrático medio entre las predicciones originales y las nuevas: {mse}")

Error cuadrático medio entre las predicciones originales y las nuevas: 5.947353057765384e-08


In [ ]:

# Comparar algunas predicciones individuales
print("\nComparación de predicciones:")
for i in range(min(5, len(y_pred_original))):
    print(f"Original: {y_pred_original[i]:.6f}, Nueva: {y_pred_new[i]:.6f}, Diferencia: {abs(y_pred_original[i] - y_pred_new[i]):.6f}")


Comparación de predicciones:
Original: 0.000383, Nueva: 0.000293, Diferencia: 0.000090
Original: 0.000383, Nueva: 0.000293, Diferencia: 0.000090
Original: 0.000383, Nueva: 0.000293, Diferencia: 0.000090
Original: 0.000383, Nueva: 0.000292, Diferencia: 0.000091
Original: 0.000383, Nueva: 0.000293, Diferencia: 0.000091


In [ ]:
# Usar el modelo de Keras directamente para comparar
y_pred_keras = model.predict(X_normalized)
y_pred_keras = scaler_y.inverse_transform(y_pred_keras).flatten()

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [ ]:
print("\nComparación con predicciones de Keras:")
for i in range(min(5, len(y_pred_original))):
    print(f"Original: {y_pred_original[i]:.6f}, Keras: {y_pred_keras[i]:.6f}, Nueva: {y_pred_new[i]:.6f}")


Comparación con predicciones de Keras:
Original: 0.000383, Keras: 0.000383, Nueva: 0.000293
Original: 0.000383, Keras: 0.000383, Nueva: 0.000293
Original: 0.000383, Keras: 0.000383, Nueva: 0.000293
Original: 0.000383, Keras: 0.000383, Nueva: 0.000292
Original: 0.000383, Keras: 0.000383, Nueva: 0.000293


In [ ]:
# Calcular el error cuadrático medio con las predicciones de Keras
mse_keras = np.mean((y_pred_original - y_pred_keras)**2)
print(f"\nError cuadrático medio entre las predicciones originales y Keras: {mse_keras}")


Error cuadrático medio entre las predicciones originales y Keras: 1.2351208758639977e-39
